In [15]:
%load_ext autoreload
%autoreload 2
import refactorings
from pathlib import Path
import difflib
import random
import pandas as pd
import random
import tqdm.notebook as tqdm
import tempfile
from contextlib import redirect_stdout
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
df = pd.read_json('/home/benja/work/weile-lab/thesis/ReVeal/data/ggnn_input/devign_cfg_full_text_files.json')
functions = df["code"].values.tolist()
print(len(functions), 'samples')

functions = functions[:10]
print('cutting to', len(functions), 'samples')

func_lines = functions[0].splitlines()
print('Example:')
print('\n'.join(func_lines[:3]))
print('...')
print('\n'.join(func_lines[-3:]))

26733 samples
cutting to 10 samples
Example:
static int tight_compress_data(VncState *vs, int stream_id, size_t bytes,
                                int level, int strategy)
 {
...
     buffer_reset(&vs->tight.zlib);
     return bytes;
 }


In [23]:
factory = refactorings.TransformationsFactory(refactorings.all_refactorings, refactorings.random_picker)
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_dir = Path(tmp_dir)
    tmp_file = tmp_dir / 'tmp.c'
    print('Temporary file:', tmp_file)
    log_filename = 'log.txt'
    print('Redirecting to', log_filename)
    new_functions = []
    with open(log_filename, 'w') as f:
        with redirect_stdout(f):
            for fn in tqdm.tqdm(functions):
                random.seed(0)
                with open(tmp_file, 'w') as f:
                    f.write(fn)
                with factory.make_project(tmp_file) as project:
                    new_file, applied = project.apply_all(return_applied=True)
                    with open(new_file) as f:
                        new_fn = f.read()
                        new_functions.append((new_fn, applied))
    with open('new_functions.pkl', 'wb') as f:
        pickle.dump(new_functions, f)

Temporary file: /tmp/tmpte7yf8qs/tmp.c
Redirecting to log.txt


In [24]:
import difflib
for i in range(len(new_functions)):
    print('Applied:', [x.__name__ for x in new_functions[i][1]])
    print(''.join(difflib.unified_diff(functions[i].splitlines(keepends=True), new_functions[i][0].splitlines(keepends=True))))

Applied: ["<class 'refactorings.insert_noop.InsertNoop'>", "<class 'refactorings.permute_stmt.PermuteStmt'>", "<class 'refactorings.rename_variable.RenameVariable'>"]
--- 
+++ 
@@ -1,8 +1,8 @@
 static int tight_compress_data(VncState *vs, int stream_id, size_t bytes,
                                 int level, int strategy)
  {
+     int fungus;
      z_streamp zstream = &vs->tight.stream[stream_id];
-     int previous_out;
      if (bytes < VNC_TIGHT_MIN_TO_COMPRESS) {
          vnc_write(vs, vs->tight.tight.buffer, vs->tight.tight.offset);
          return bytes;
@@ -18,14 +18,15 @@
      zstream->next_out = vs->tight.zlib.buffer + vs->tight.zlib.offset;
      zstream->avail_out = vs->tight.zlib.capacity - vs->tight.zlib.offset;
      zstream->data_type = Z_BINARY;
-     previous_out = zstream->total_out;
+     char * mungus = "hello";
+     fungus = zstream->total_out;
      /* start encoding */
      if (deflate(zstream, Z_SYNC_FLUSH) != Z_OK) {
          fprintf(stderr, "VNC: erro